In [6]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("ggplot")
from cycler import cycler

- https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html
- https://matplotlib.org/stable/tutorials/colors/colors.html
- https://matplotlib.org/stable/tutorials/text/annotations.html#plotting-guide-annotation

## Chainage plot tool

## User settings
The following parameters need to be set by the user

In [7]:
# Mandatory
PLOT_OUTPUTS_FC = r"revisedGIS1DO_9924_total"
RIVER_FC = r"River_Serpentine"
DETERMINAND = "DO"

# Styling
TARGET_FLAG = False  # True/False
TARGET_ANNOTATE = True  # True/False
ANNOTATE_FILTER = 0  # mg/l diff (Set to 0 for no filter)
FIG_SIZE = 12
ARROW_LENGTH = 0.5
LEGEND_LOC = "upper right"  # Supported legend values are: 'best', 'upper right', 'upper left',
# 'lower left', 'lower right', 'right', 'center left', 'center right',
# 'lower center', 'upper center', 'center'

### Code

In [8]:
# Carry out spatial selection on plot outputs using the river (2m intersection distance)
arcpy.management.SelectLayerByLocation(
    PLOT_OUTPUTS_FC, "INTERSECT", RIVER_FC, "2 Meters", "NEW_SELECTION", "NOT_INVERT"
)

# Read feature class as dataframe
df = pd.DataFrame.spatial.from_featureclass(PLOT_OUTPUTS_FC)

In [9]:
columns = [
    "OBJECTID",
    "ReachNo",
    "MeanConc",
    "LCLimMnCon",
    "UCLimMnCon",
    "CalcValQ90",
    "LCLimPer90",
    "UCLimPer90",
    "CalcValQ95",
    "LCLimPer95",
    "UCLimPer95",
    "CalcValQ99",
    "LCLimPer99",
    "UCLimPer99",
    "FeatName",
    "US_DS_Feat",
    "ReachName",
    "ObsConc",
    "ObsConcUCL",
    "ObsConcLCL",
    "ObsQ90Conc",
    "ObsQ90ConcUCL",
    "ObsQ90ConcLCL",
    "ObsQ95Conc",
    "ObsQ99Conc",
    "TargetMean",
    "Target90",
    "DetNo",
    "SWConc",
    "IMConc",
    "INConc",
    "MIConc",
    "LSConc",
    "ARConc",
    "HWConc",
    "URConc",
    "ATConc",
    "BGConc",
    "STConc",
    "LKConc",
    "DConc",
    "TargetHigh",
    "TargetGood",
    "TargetMod",
    "TargetPoor",
    "EA_WB_ID",
    "DiffConc",
    "PntConc",
    "CalStatus",
    "CalScore",
    "DISHeadKM",
    "DISPOINTKM",
]

# Filter necessary columns
df = df[columns].copy()

# Sort per reach ascending
df.sort_values(["ReachNo", "OBJECTID"], inplace=True)

# Calculate cummulative distance
df["DISTANCE"] = df["DISPOINTKM"].cumsum()

# Calculate diff in concentration
df["DIFF_CONC"] = df["MeanConc"].diff().fillna(0)

# Remove first point next to headwater
df = df[1:].copy()

# Set distance as index for x axis
df.set_index("DISTANCE", inplace=True)

# Get rid of 0s in Observed concentrations
df["ObsConc"].replace(0, np.NaN, inplace=True)
df["ObsConcLCL"].replace(0, np.NaN, inplace=True)
df["ObsConcUCL"].replace(0, np.NaN, inplace=True)

# Rename sectors for a better legend
apportionment_cols = {
    "SWConc": "Sewage",
    "IMConc": "Intermittent",
    "INConc": "Industry",
    "MIConc": "Mines",
    "LSConc": "Livestock",
    "ARConc": "Arable",
    "HWConc": "Highways",
    "URConc": "Urban",
    "ATConc": "Atmospheric",
    "BGConc": "Background",
    "STConc": "Septic tanks",
    "LKConc": "Lakes",
}

# Add reach diffuse if needed
if DETERMINAND in ["Ammonia", "BOD", "DO"]:
    apportionment_cols["DiffConc"] = "Reach Diffuse"

df.rename(apportionment_cols, inplace=True, axis=1)

In [10]:
# Set figure size
plt.rcParams["figure.figsize"] = (FIG_SIZE, FIG_SIZE / (1.618 * 2.5 / 3))

# Prepare colorscale
apportionment_colormap = cycler(
    "color",
    [
        "0.1",
        "0.5",
        "r",
        "b",
        "yellowgreen",
        "yellow",
        "g",
        "purple",
        "olive",
        "pink",
        "gold",
        "orange",
        "lightsteelblue"
    ],
)
plt.rcParams["axes.prop_cycle"] = apportionment_colormap

# Set up initial layout
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True)

## Calibration plot
# Simulated data
df.MeanConc.plot(ax=axes[0], c="k")
df.UCLimMnCon.plot(ax=axes[0], c="k", ls="--", alpha=0.75)
df.LCLimMnCon.plot(ax=axes[0], c="k", ls="--", alpha=0.75)

# Observed data
df.ObsConc.plot(ax=axes[0], c="b", marker="o")
df.ObsConcUCL.plot(ax=axes[0], c="b", marker=".")
df.ObsConcLCL.plot(ax=axes[0], c="b", marker=".")

## Apportionment plot
# Data
df[apportionment_cols.values()].replace(0, np.NaN).plot.area(ax=axes[1], lw=0)

# Targets
if TARGET_FLAG:
    df.TargetPoor.plot(ax=axes[1], c="r", ls="--", alpha=0.5)
    df.TargetMod.plot(ax=axes[1], c="orange", ls="--", alpha=0.5)
    df.TargetGood.plot(ax=axes[1], c="g", ls="--", alpha=0.5)
    df.TargetHigh.plot(ax=axes[1], c="b", ls="--", alpha=0.5)

# Titles
fig.suptitle(
    f"{DETERMINAND} calibration (top) and apportionment (bottom) for reaches {df.ReachNo.min()} to {df.ReachNo.max()}",
    size=12,
    ha="center",
    y=0.98,
)
# axes[0].set_title("Calibration")
# axes[1].set_title("Apportionment")

# Make sure axis starts at 0
axes[0].set_ylim(bottom=0)
axes[1].set_ylim(bottom=0)

# Axis labels
axes[0].set_ylabel("Concentration (mg/L)")
axes[1].set_ylabel("Concentration (mg/L)")
axes[1].set_xlabel("Distance (km)")

# Activate legend
axes[0].legend(
    labels=[
        "Sim. Mean",
        "Sim. Mean UCL",
        "Sim. Mean LCL",
        "Obs. Mean",
        "Obs. Mean UCL",
        "Obs. Mean LCL",
    ]
)
axes[1].legend(ncol=2, loc=LEGEND_LOC)  # change to 'upper right' if needed

# Annotations
if TARGET_ANNOTATE:
    # Apply filter
    mask = (df["DIFF_CONC"] > ANNOTATE_FILTER) & (df.US_DS_Feat == "d-s")
    # Draw labels
    count = 0
    for dis, row in df[mask].iterrows():
        axes[1].annotate(
            row.FeatName,
            xy=(dis, row.MeanConc),
            xycoords="data",
            xytext=(dis, row.MeanConc + ARROW_LENGTH + 0 * (count % 2)),
            textcoords="data",
            va="top",
            ha="center",
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color="k"),
            bbox=dict(boxstyle="square, pad=0.3", fc="1", ec="0.5", lw=1, alpha=0.5),
        )
        count += 1

# Make everything tidy
plt.tight_layout()

# Create Figures folder in project folder if it doesn't exist
p = arcpy.mp.ArcGISProject("CURRENT")
fig_folder = os.path.join(p.homeFolder, "Figures")
if not os.path.exists(fig_folder):
    os.mkdir(fig_folder)

# Save figure and show
fig_name = f"{DETERMINAND}_outputs_{df.ReachNo.min()}_{df.ReachNo.max()}.png"
fig_path = os.path.join(fig_folder, fig_name)
plt.savefig(fig_path)
plt.show()

**TODO**
- Transform into a function for batch run
- Could be interesting to plot cal score
- Could also annotate EA_WB_ID and confluences